## 9. 朴素贝叶斯：文本分类永恒的经典
### 9.1 理解朴素贝叶斯

朴素贝叶斯，在文本分类上它是一个非常靠谱的基准(baseline)。  
朴素贝叶斯-核心思想：统计单词在不同类别中出现的概率，然后根据这些结果进一步判断一个文本它于不同类别的概率。  
贝叶斯定理：  
$$p(y|x) = \frac{p(x|y)p(y)}{p(x)}$$  
- p(y|x) 后验概率 posterior
- p(x|y) 似然(likelihood)
- p(y) 先验概率(prior)
- p(x) Normalization

条件独立的假设 计算 联合概率   
条件独立(conditional independence)的假设，是为什么我们把朴素贝叶斯说成“朴素”的主要的原因。  

注意：朴素贝叶斯法和贝叶斯估计是不同的概念。

潜在风险：比如其中一个概率等于0， 那不管其他概率值是多少，最后的结果一定会等于0。  
处理方法：平滑处理。   
常用的方法- 加1平滑 add-one smoothing：
$$p(w|y=c) = \frac{类别为C的语料库中单词w出现的次数 + 1}{类别为C的语料库中包含的所有单词的个数 + v}$$
v为词典的大小，即出现单词的种类。  

例如：垃圾邮件分类。


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as malb

# read data
df = pd.read_csv('spam.csv',encoding = 'latin')# ,error_bad_lines=False
df.head()

# rename v1,v2 ,使得拥有更好的可读性
df.rename(columns={'v1':'Label','v2':'Text'},inplace=True) # 是否返回一个新的DataFrame。如果为真，则忽略copy的值。
df.head()

# 把 ham spam  one-hot
df['numLabel'] = df['Label'].map({'ham':0,'spam':1})
df.head()

# 统计有多少个ham,有多少个spam
# print("The number of ham:",len(df[df.numLabel == 0]),"The number of spam:",len(df[df.numLabel == 1]))
# print("The number of total simples:",len(df)) 

# 统计文本长度信息 并画出 histogram
# text_lengths = [len(df.loc[i,'Text']) for i in range(len(df))] # df.loc[i,'Text']选取标签为Text的列
# plt.hist(text_lengths,200,facecolor='blue',alpha=0.5)
# plt.xlim([0,200])
# plt.show()

#导入英文的停用词库
from sklearn.feature_extraction.text import CountVectorizer
# CountVectorizer 类会将文本中的词语转换为词频矩阵

# 构建文本的向量（基于词频的表示）
vectorizer = CountVectorizer() # 创建词袋数据结构
X = vectorizer.fit_transform(df.Text) # fit_transform函数计算各个词语出现的次数。
y = df.numLabel

# 将数据分成训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=100)
print("训练数据中的样本个数：",X_train.shape[0],"测试数据中的样本个数：",X_test.shape[0])

# 利用朴素贝叶斯做训练
from sklearn.naive_bayes import MultinomialNB # 先验为多项式分布的朴素贝叶斯
from sklearn.metrics import accuracy_score

model = MultinomialNB(alpha=1.0,fit_prior=True)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Accurancy on test data:",accuracy_score(y_test,y_pred))

# 打印混淆矩阵
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred, labels=[0,1])

训练数据中的样本个数： 4457 测试数据中的样本个数： 1115
Accurancy on test data: 0.97847533632287


array([[956,  14],
       [ 10, 135]])

![image.png](Naive_bayes.png)

### 9.2 最大似然以及参数估计
- 无条件优化
- 带约束条件的优化

**拉格朗日乘数法（Lagrange multiplier）**   
传统几何方法，现在使用拉格朗日乘数法
![image.png](Lagrange_multiplier.png)

### 构造朴素贝叶斯目标函数己求出最优解
#### 1. 构造朴素贝叶斯的最大似然估计
$$D{(x_1,y_1),(x_2,y_2),(x_3,y_3)\dots (x_n,y_n)}$$
$x特征,y标签$  x可能是离散型，也可能是连续型。文本每一个特征是单词可以认为这些特都是**离散型**的。  
$$argmax\ p(D) = argmax\ \prod_{i=1}^{n} p(x_i,y_i) = argmax\ \prod_{i=1}^{n} p(y_i)p(x_i|y_i)$$
独立分布概率 --> 联合分布概率 ;$p(y_i)$每个类别的先验概率  

判别模型 生成模型  

首先，构建一个**词库**。文本-->向量；每个向量的大小 == 词库的大小。   
因为$p(x_1|y_1) = \prod_{j=1}^{v} p(w_j|y_1)^{x_{1j}}$ ,其中n为样本数，V为词库大小。  
So,$$argmax\ \prod_{i=1}^{n} p(y_i)p(x_i|y_i) = argmax\ \prod_{i=1}^{n} p(y_i) \prod_{j=1}^{v} p(w_j|y_i)^{x_{ij}}$$  

在argmax的时候加log不改变优化目标。

#### 2. 优化： 。。。。。。

### 9.3 高斯朴素贝叶斯
连续型特征： 比如一个人的身高值、温度、年龄。  
如果特征是连续型的，我们则可以使用高斯分布来描述此条件概率。  
- 高斯分布具有很多优点，比如在计算层面上有很多漂亮的特点(两个高斯分布相加/乘积是高斯分布，两个高斯分布的条件概率也是高斯分布等等)，这些特点会让计算变得尤其简单。  
- 高斯分布特别适合表示真实的世界。其中最经典的定理叫作**中心极限定理**：只要我们观测到足够多的数据，那这些数据会服从高斯分布。  
  
针对一个特征在某一个类别(y=0,1,...)中的概率。利用某个类别下某个特征的所有连续型数据来拟合一个高斯分布，再去预测。   

实际的工作当中，如果数据中有大量的连续型特征，我们其实是不会使用朴素贝叶斯模型的，而是使用逻辑回归、XGBoost等模型。   
总结起来，朴素贝叶斯本身**最适合解决文本分类问题**。所以之后遇到任何一个只要跟文本相关的问题，可以尝试使用朴素贝叶斯来解决，至少它可以作为很靠谱的基准。

#### 补充说明高斯分布
$$X\sim N(\mu,\sigma^2)$$  
其中$\mu期望；\sigma^2 方差$。

则其概率密度函数为:

$$f(x) = {1 \over \sigma\sqrt{2\pi} }\,e^{- {{(x-\mu )^2 \over 2\sigma^2}}}$$


Reference:  
推荐一本书[《正态分布的前世今生》](http://wise.xmu.edu.cn/_data/2016/07/28/c6a3daf3_e3c2_49e3_9aad_bf95fa4446bf/%E6%AD%A3%E5%A4%AA%E5%88%86%E5%B8%83%20final.pdf)   
wikipedia:[正态分布](https://zh.wikipedia.org/wiki/正态分布)
  

### 9.4 生成模型与判别模型
**生成模型:** 顾名思义，生成模型是可以用来生成样本的！  
例如：**GAN**，让机器写程序，让机器学会画画，让机器给自己编一个曲子，这就是生成模型可以做的事情。  

**判别模型:** 主要用来判断样本的类别。

| 生成模型       | 判别模型     |
| -------------- | ------------ |
| 朴素贝叶斯     | 逻辑回归     |
| 隐马尔科夫模型 | 支持向量机   |
| 高斯混合模型   | 条件随意场   |
| 主题模型       | 卷积神经网络 |

### 9.5 小结
[朴素贝叶斯](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes)一共有三种方法，分别是高斯分布(Gaussian naive Bayes)、多项分布(Multinomial naive Bayes)、伯努利分布(Bernoulli naive Bayes)。  
这三种分类方法其实就是对应三种不同的数据分布类型。

多项式分布（Multinomial Distribution）是二项式分布的推广，二项分布是随机结果值只有两个(投硬币的结果)，多项式分布是指随机结果值有多个(摇骰子的结果)。

多项式模型朴素贝叶斯和伯努利模型朴素贝叶斯常用在文本分类问题中，高斯分布的朴素贝叶斯主要用于连续变量中,且假设连续变量是**服从正太分布的**。


#### 高斯朴素贝叶斯
高斯朴素贝叶斯算法是假设特征的可能性(即概率)为高斯分布。

> class sklearn.naive_bayes.GaussianNB(priors=None)

`priors`:先验概率大小，如果没有给定，模型则根据样本数据自己计算（利用极大似然法）。

##### 对象  
`class_prior_`:每个样本的概率

`class_count`:每个类别的样本数量

`theta_`:每个类别中每个特征的均值

`sigma_`:每个类别中每个特征的方差

#### [多项式分布贝叶斯](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)
适用于服从多项分布的特征数据。

>class sklearn.naive_bayes.MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

`alpha`:先验平滑因子，默认等于1，当等于1时表示拉普拉斯(Laplace)平滑，小于1表示Lidstone平滑。

`fit_prior`:是否去学习类的先验概率，默认是True. 布尔型可选参数，默认为True。布尔参数fit_prior表示是否要考虑先验概率，如果是false，则所有的样本类别输出都有相同的类别先验概率。否则可以自己用第三个参数class_prior输入先验概率，或者不输入第三个参数class_prior，让MultinomialNB自己从训练集样本来计算先验概率，此时的先验概率为$P(Y=C_k)=m_k/m$其中m为训练集样本总数量，$m_k$为输出为第k类别的训练集样本数。

`class_prior`:各个类别的先验概率，如果没有指定，则模型会根据数据自动学习， 每个类别的先验概率相同，等于类标记总个数N分之一。

| fit_prior | class_prior      | 最终先验概率           |
| --------- | :----------------: | :--------------------------------: |
| False     | 填或不填没有意义 | $P(Y=C_k)=1/k$         |
| True      | 不填             | $P(Y=C_k)=m_k/m$       |
| True      | 填               | $P(Y=C_k)=class\_prior$ |

##### 对象
`class_log_prior_`:每个类别平滑后的先验概率

`intercept_`:是朴素贝叶斯对应的线性模型，其值和class_log_prior_相同feature_log_prob_:给定特征类别的对数概率(条件概率)。 特征的条件概率=（指定类下指定特征出现的次数+alpha）/（指定类下所有特征出现次数之和+类的可能取值个数*alpha）coef_: 是朴素贝叶斯对应的线性模型，其值和feature_log_prob相同

`class_count_`: 训练样本中各类别对应的样本数

`feature_count_`: 每个类别中各个特征出现的次数

##### Methods
`fit(self, X, y[, sample_weight])`:	在数据集(X,Y)上拟合模型。

`get_params(self[, deep])`:	获取模型参数。

`partial_fit(self, X, y[, classes, sample_weight])`:	Incremental fit on a batch of samples.

`predict(self, X)`:	对数据集X进行预测。

`predict_log_proba(self, X)`: 对数据集X预测，得到每个类别的概率对数值。

`predict_proba(self, X)`: 对数据集X预测，得到每个类别的概率。

`score(self, X, y[, sample_weight])`: 得到模型在数据集(X,Y)的得分情况。

`set_params(self, \*\*params)`:	Set the parameters of this estimator.

#### 伯努利朴素贝叶斯
用于多重伯努利分布的数据，即有多个特征，但每个特征都假设是一个二元 (Bernoulli, boolean) 变量。

> class sklearn.naive_bayes.BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)
`alpha`:平滑因子，与多项式中的alpha一致。

`binarize`:样本特征二值化的阈值，默认是0。如果不输入，则模型会认为所有特征都已经是二值化形式了；如果输入具体的值，则模型会把大于该值的部分归为一类，小于的归为另一类。

`fit_prior`:是否去学习类的先验概率，默认是True

`class_prior`:各个类别的先验概率，如果没有指定，则模型会根据数据自动学习， 每个类别的先验概率相同，等于类标记总个数N分之一。

##### 对象
`class_log_prior_`:每个类别平滑后的先验对数概率。

`feature_log_prob_`:给定特征类别的经验对数概率。

`class_count_`:拟合过程中每个样本的数量。

`feature_count_`:拟合过程中每个特征的数量。

##### 方法
贝叶斯的方法和其他模型的方法一致。

`fit(X,Y)`:在数据集(X,Y)上拟合模型。

`get_params()`:获取模型参数。

`predict(X)`:对数据集X进行预测。

`predict_log_proba(X)`:对数据集X预测，得到每个类别的概率对数值。

`predict_proba(X)`:对数据集X预测，得到每个类别的概率。

`score(X,Y)`:得到模型在数据集(X,Y)的得分情况。